In [272]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [273]:
dataset = pd.read_excel('../data/boxScores.xlsx')


In [274]:
# Filter out rows where the 'MIN' column is zero
dataset = dataset[dataset['MIN'] != 0]
dataset = dataset.dropna(subset=['W/L'])
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,PF,+/-,FP,Last3_FP_Avg,Last3_Min,Last5_FP_Avg,Last5_Min,Last7_FP_Avg,Last7_Min,Season_FP_Avg
count,9982,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,...,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000,9982.000000
mean,2024-11-22 14:11:51.200160256,22.482068,10.529253,3.836806,8.263474,1.261170,3.502104,1.594470,2.046584,1.012222,...,1.785514,-0.000701,21.593288,21.480266,22.362369,21.392051,22.268400,21.322921,22.201792,21.572414
min,2024-10-22 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-43.000000,-3.000000,-1.000000,0.666667,-1.000000,1.000000,-1.000000,1.000000,-1.000000
25%,2024-11-06 00:00:00,14.000000,3.000000,1.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,-7.000000,9.700000,11.437500,14.666667,11.661667,14.800000,11.728571,14.723214,12.375000
50%,2024-11-21 12:00:00,24.000000,9.000000,3.000000,7.000000,1.000000,3.000000,1.000000,1.000000,1.000000,...,2.000000,0.000000,19.800000,20.066667,23.333333,20.000000,23.200000,19.900000,23.285714,20.036364
75%,2024-12-08 00:00:00,31.000000,16.000000,6.000000,12.000000,2.000000,5.000000,2.000000,3.000000,2.000000,...,3.000000,7.000000,31.200000,29.891667,30.666667,29.675000,30.600000,29.480357,30.428571,28.755172
max,2024-12-28 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,6.000000,43.000000,98.300000,80.500000,44.666667,75.360000,44.000000,72.200000,44.000000,63.355556
std,NaN,10.847488,8.817063,3.234562,6.034359,1.537369,3.171918,2.246381,2.715058,1.343533,...,1.452097,11.310923,14.981087,13.138480,10.047682,12.732902,9.865021,12.570748,9.792995,11.901225


In [275]:
# Filter out players who aren't in clusters for accuracy
clusterdf = pd.read_excel('../data/clusteredPlayers.xlsx')
clusterdf = clusterdf.drop('Unnamed: 0', axis=1)
clusterDict = {}
clusterDict.update(pd.Series(clusterdf['Cluster'].values, index=clusterdf['Player']).to_dict())
dataset['CLUSTER'] = dataset['PLAYER'].map(clusterDict).astype('category')
dataset = dataset.dropna(subset=['CLUSTER'])
dataset.sort_values(by=['GAME DATE'], ascending = [False], inplace=True)
len(dataset)

7588

In [276]:
dataset.describe()

,GAME DATE,MIN,PTS,FGM,FGA,3PM,3PA,FTM,FTA,OREB,...,PF,+/-,FP,Last3_FP_Avg,Last3_Min,Last5_FP_Avg,Last5_Min,Last7_FP_Avg,Last7_Min,Season_FP_Avg
count,7588,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,...,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000
mean,2024-11-22 15:28:45.039536384,26.424354,12.698339,4.619267,9.902214,1.523195,4.190828,1.936610,2.463627,1.158540,...,2.042699,0.188719,25.732275,25.612485,26.309787,25.532406,26.227713,25.471457,26.169713,25.725504
min,2024-10-22 00:00:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-43.000000,-2.800000,-1.000000,1.000000,-1.000000,1.000000,-1.000000,1.000000,8.242105
25%,2024-11-06 00:00:00,20.000000,6.000000,2.000000,6.000000,0.000000,2.000000,0.000000,0.000000,0.000000,...,1.000000,-8.000000,15.200000,16.900000,21.000000,17.080000,20.800000,17.162500,20.800000,17.841935
50%,2024-11-22 00:00:00,27.000000,11.000000,4.000000,9.000000,1.000000,4.000000,1.000000,2.000000,1.000000,...,2.000000,0.000000,24.400000,24.133333,26.666667,24.130000,26.600000,24.042857,26.571429,23.884000
75%,2024-12-08 00:00:00,33.000000,18.000000,7.000000,13.000000,2.000000,6.000000,3.000000,4.000000,2.000000,...,3.000000,8.000000,34.500000,32.966667,32.333333,32.540000,32.250000,32.368750,32.142857,32.138462
max,2024-12-28 00:00:00,48.000000,60.000000,22.000000,39.000000,10.000000,20.000000,17.000000,26.000000,12.000000,...,6.000000,43.000000,98.300000,80.500000,44.666667,75.360000,44.000000,72.200000,44.000000,63.355556
std,NaN,8.576619,8.736605,3.209545,5.798668,1.615829,3.220258,2.395641,2.876413,1.416461,...,1.433108,12.287287,14.167288,11.917407,7.548196,11.403778,7.305947,11.190193,7.201513,10.318794


For reference, features are the raw input data provided to the model to generate predictions, while the label is the target outcome that the model aims to predict. These above features can and probably should be tweaked and adjusted to improve the model's performance.

In [277]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfLabels = dataset[labelName]

In [254]:
dfLabels = dataset[labelName]
dfLabels.head()

,FP
1687,17.9
1472,29.5
6962,29.2
3527,23.2
6985,22.1


In [278]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)

** Note numpy array is only numbers while pandas dataframe can be a mix of numbers and strings.

In [279]:
# Scaling the data
x = features
x = StandardScaler().fit_transform(x)

In [280]:
dataset = dataset[['PLAYER', 'GAME DATE', 'FP', 'Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']]
players = dataset['PLAYER']
gameDates = dataset['GAME DATE']

In [281]:
featureNames = ['Last3_FP_Avg', 'Last5_FP_Avg', 'Last7_FP_Avg', 'Season_FP_Avg']
labelName = ['FP']
dfFeatures = dataset[featureNames]
dfFeatures.head()

In [282]:
dfLabels = dataset[labelName]
dfLabels.head()

,FP
1687,17.9
1472,29.5
6962,29.2
3527,23.2
6985,22.1


In [283]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)
train, test, trainLabels, testLabels, trainPlayers, testPlayers, trainDates, testDates= train_test_split(features, labels, players, gameDates, test_size=0.2, random_state=4)
testPlayers = testPlayers.reset_index(drop=True)
testDates = testDates.reset_index(drop=True)


In [284]:
rf = RandomForestRegressor(random_state=4)

In [285]:
rf.fit(train, trainLabels)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(random_state=4)

In [286]:
rfPredNoStandard = rf.predict(test)

In [287]:
df3 = pd.DataFrame(test, columns = featureNames)
df3['PLAYER'] = testPlayers
df3['GAME DATE'] = testDates
df3['actual'] = testLabels
df3['predicted'] = rfPredNoStandard
df3['error'] = abs(df3['actual'] - df3['predicted'])

In [288]:
# 3 isn't standardized which makes featurenames more readable
# Reorder columns to make PLAYER the first column
df3 = df3[['PLAYER'] + [col for col in df3.columns if col != 'PLAYER']]
df3.head()

,PLAYER,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,GAME DATE,actual,predicted,error
0,Jusuf Nurkić,32.900000,24.84,22.757143,25.534783,2024-11-27,14.9,37.990,23.090
1,Haywood Highsmith,17.166667,19.18,19.242857,16.114815,2024-11-17,4.3,17.502,13.202
2,De'Aaron Fox,38.166667,36.08,39.000000,44.334375,2024-11-11,37.7,36.797,0.903
3,Isaiah Joe,21.566667,21.04,21.285714,15.910714,2024-11-15,26.0,21.071,4.929
4,Tyrese Haliburton,41.700000,42.04,38.671429,38.012500,2024-11-18,45.6,43.237,2.363


In [289]:
df3.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,GAME DATE,actual,predicted,error
count,1518.000000,1518.000000,1518.000000,1518.000000,1518,1518.000000,1518.000000,1518.000000
mean,25.572705,25.415233,25.345492,25.582488,2024-11-22 08:30:21.343873536,25.526416,25.670377,6.549256
min,0.000000,0.000000,0.000000,8.242105,2024-10-22 00:00:00,-2.000000,0.106000,0.001000
25%,16.800000,16.850000,17.300000,17.764286,2024-11-06 00:00:00,14.800000,16.661750,2.327000
50%,23.800000,23.865000,23.850000,23.672581,2024-11-21 00:00:00,23.800000,23.894000,5.330000
75%,32.891667,32.600000,32.192857,32.186667,2024-12-07 00:00:00,34.700000,33.699750,9.484500
max,80.500000,75.360000,69.142857,63.355556,2024-12-28 00:00:00,87.800000,81.242000,41.290000
std,12.063759,11.448347,11.178369,10.327648,NaN,14.618532,12.286061,5.452378


In [290]:
train, test, trainLabels, testLabels = train_test_split(x, labels, test_size=0.2, random_state=4)
rf = RandomForestRegressor(random_state=4)
rf.fit(train, trainLabels)
rfPredNoStandard = rf.predict(test)

c:\Users\Ian Bracken\Desktop\WebDev\nba-prediction-model\venv\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [291]:
df4 = pd.DataFrame(test, columns = featureNames)
df4['actual'] = testLabels
df4['predicted'] = rfPredNoStandard
df4['error'] = abs(df4['actual'] - df4['predicted'])

In [292]:
df4.describe()

,Last3_FP_Avg,Last5_FP_Avg,Last7_FP_Avg,Season_FP_Avg,actual,predicted,error
count,1518.000000,1518.000000,1518.000000,1518.000000,1518.000000,1518.000000,1518.000000
mean,-0.003338,-0.010276,-0.011257,-0.013861,25.526416,25.670400,6.553842
std,1.012347,1.003974,0.999009,1.000924,14.618532,12.286451,5.445597
min,-2.149308,-2.239090,-2.276380,-1.694437,-2.000000,0.106000,0.015000
25%,-0.739512,-0.761412,-0.730282,-0.771577,14.800000,16.641750,2.355000
50%,-0.152097,-0.146225,-0.144909,-0.198963,23.800000,23.863000,5.323500
75%,0.610843,0.619800,0.600691,0.626196,34.700000,33.706000,9.497750
max,4.605963,4.369681,3.902907,3.646989,87.800000,81.242000,41.290000


In [293]:
pd.DataFrame.to_excel(df3, '../data/playerPredictions.xlsx')

In [294]:
dfCheck = pd.DataFrame()
# Difference in accuracy between df3 and df4 is negligible so I am sticking with 3 for better readability
dfCheck['df3'] = df3['error']
dfCheck['df4'] = df4['error']
dfCheck.describe()

,df3,df4
count,1518.000000,1518.000000
mean,6.549256,6.553842
std,5.452378,5.445597
min,0.001000,0.015000
25%,2.327000,2.355000
50%,5.330000,5.323500
75%,9.484500,9.497750
max,41.290000,41.290000


Note: Scaling and preprocessing data matters more on larger datasets